In [7]:
from ultralytics import YOLO
from os import path, listdir, makedirs
from shutil import rmtree, copy
import numpy as np
import time
import random
import cv2

### Train the full classifer

In [8]:
# Constants
IMAGE_SIZE = 640 # x 480
PROJECT_NAME = "logs"
RECIPE_NAME = "all"
TRAIN_RUN = f"{RECIPE_NAME}_run-{time.strftime('%Y%m%d-%H%M%S')}"
DATA_PATH = "./datasets/full"

# Model
MODEL_PATH = f"./models/pretrained/yolov8n-obb-dotav1.pt"
TRAIN_PARAM = {
    # Model definition
    'data': f"./models/recipes/dataset_desc.yaml",
    'resume': False,
    'device': '0',
    'pretrained': True,
    # Names
    'project' : PROJECT_NAME,
    'name': TRAIN_RUN,
    # Training Parameters
    'batch': -1,
    'imgsz': IMAGE_SIZE,
    'epochs': 20,
    'patience': 10,
    'cos_lr': True,
    # Augmentation
    'hsv_h': 0.05, # Higher than default for resistor
    'hsv_s': 0.3, # Colours should not change too much
    'hsv_v': 0.2, # Colours should not change too much
    'degrees': 180, # Rotation
    'translate': 0.1, # Translation
    'scale': 0.8, # Scaling - camera is always at the same distance
    'shear': 10.0, # Shearing
    'perspective': 0.0, # Perspective
    'flipud': 0.5, # Flip up-down
    'fliplr': 0.5, # Flip left-right
    'mosaic': 0.5, # Mosaic
    'mixup': 0.0, # Mixup
    'copy_paste': 0.0, # Copy-paste
    'crop_fraction': 1.0, # Crop fraction
    # Loss weights
    'cls' : 1.0, # Class
    'box' : 4.0, # Box accuracy
    'dfl' : 1.5, # Help manage unbalanced classes
    # Post parameters
    'save': True,
    'save_period': 5,
    'plots': False,
    # Misc
    'verbose': False,
}

PATHS = {
    'train': f"{DATA_PATH}/current/images/train",
    'val': f"{DATA_PATH}/current/images/val",
    'test': f"{DATA_PATH}/current/images/test",
    'resistors' : f"{DATA_PATH}/resistor/imgs",
    'ceramic_cap' : f"{DATA_PATH}/ceramic_capacitor/imgs",
}


In [3]:
# Reorganise the data
PARAM = {
    'path' : DATA_PATH,
    'train' : 0.7,
    'val' : 0.2,
    'test' : 0.1,
    'include' : [
        'resistor',
        'capacitor',
        'ceramic_capacitor',
        'film_capacitor',
        'inductor',
        'led',
        'wire'
    ]
}
# Remove old dataset
DATASET_FOLDER = f"{DATA_PATH}/current"
if path.exists(DATASET_FOLDER):
    for folder in listdir(DATASET_FOLDER):
        rmtree(path.join(DATASET_FOLDER, folder), ignore_errors=True)
    # Make label folder
    makedirs(path.join(DATASET_FOLDER, 'labels'), exist_ok=True)
    # Make train, val, test folders
    for folder in ['train', 'val', 'test']:
        makedirs(path.join(DATASET_FOLDER, 'images', folder), exist_ok=True)
        makedirs(path.join(DATASET_FOLDER, 'labels', folder), exist_ok=True)

# Get all component images from all folders
basenames = []
# For component folder in dataset
for folder in listdir(PARAM['path']):
    # Skip if not in include
    if folder not in PARAM['include']: continue
    # For each subfolder in component folder
    imgfiles = listdir(path.join(PARAM['path'], folder, 'imgs'))
    labfiles = listdir(path.join(PARAM['path'], folder, 'labels'))
    bases = [path.join(folder, 'imgs', path.splitext(f)[0]) for f in imgfiles]
    basenames.extend(bases)
    print(f"Found {len(imgfiles)} images and {len(labfiles)} labels in {folder}")

# Split the data into train, val, test
random.shuffle(basenames)
train = int(len(basenames) * PARAM['train'])
val = int(len(basenames) * PARAM['val'])
test = int(len(basenames) * PARAM['test'])
print(f"Split into {train} train, {val} val, {test} test. Total: {train+val+test} images.")
train_set = basenames[:train]
val_set = basenames[train:train+val]
test_set = basenames[train+val:]

# Copy the images and labels to the new dataset folder
for folder, dataset in zip(['train', 'val', 'test'], [train_set, val_set, test_set]):
    for base in dataset:
        filename = path.split(base)[1]
        copy(path.join(PARAM['path'], f"{base}.png"), path.join(DATASET_FOLDER, 'images', folder, f"{filename}.png"))
        base = base.replace('imgs', 'labels')
        copy(path.join(PARAM['path'], f"{base}.txt"), path.join(DATASET_FOLDER, 'labels', folder, f"{filename}.txt"))


Found 164 images and 164 labels in capacitor
Found 264 images and 264 labels in ceramic_capacitor
Found 66 images and 66 labels in film_capacitor
Found 52 images and 52 labels in inductor
Found 96 images and 96 labels in led
Found 258 images and 259 labels in resistor
Found 75 images and 75 labels in wire
Split into 682 train, 195 val, 97 test. Total: 974 images.


In [6]:
# Tensorboard logging
%load_ext tensorboard
%tensorboard --logdir "logs" --port=6005

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


Reusing TensorBoard on port 6005 (pid 25652), started 13:30:46 ago. (Use '!kill 25652' to kill it.)

In [21]:
! taskkill /PID 40156 /F

ERROR: The process "40156" not found.


In [10]:
# Train
model = YOLO(MODEL_PATH).to('cuda')
model.train(**TRAIN_PARAM)

Ultralytics YOLOv8.2.18  Python-3.10.13 torch-2.1.1+cu121 CUDA:0 (NVIDIA GeForce RTX 3080 Laptop GPU, 16384MiB)
engine\trainer: task=obb, mode=train, model=./models/pretrained/yolov8n-obb-dotav1.pt, data=./models/recipes/dataset_desc.yaml, epochs=20, time=None, patience=10, batch=-1, imgsz=640, save=True, save_period=5, cache=False, device=0, workers=8, project=logs, name=all_run-20240521-124737, exist_ok=False, pretrained=True, optimizer=auto, verbose=False, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=True, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=False, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False

train: Scanning C:\Users\Shaheen\OneDrive - Imperial College London\Uni\CW Labs\Year 4\FYP\src\vision\datasets\current\labels\train.cache... 682 images, 0 backgrounds, 0 corrupt: 100%|██████████| 682/682 [00:00<?, ?it/s]
val: Scanning C:\Users\Shaheen\OneDrive - Imperial College London\Uni\CW Labs\Year 4\FYP\src\vision\datasets\current\labels\val.cache... 195 images, 0 backgrounds, 0 corrupt: 100%|██████████| 195/195 [00:00<?, ?it/s]


optimizer: 'optimizer=auto' found, ignoring 'lr0=0.01' and 'momentum=0.937' and determining best 'optimizer', 'lr0' and 'momentum' automatically... 
optimizer: AdamW(lr=0.000667, momentum=0.9) with parameter groups 63 weight(decay=0.0), 73 weight(decay=0.0003515625), 72 bias(decay=0.0)
TensorBoard: model graph visualization added 
Image sizes 640 train, 640 val
Using 8 dataloader workers
Logging results to logs\all_run-20240521-124737
Starting training for 20 epochs...

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


       1/20      6.47G      1.057       11.1      2.472         13        640: 100%|██████████| 16/16 [00:06<00:00,  2.57it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 5/5 [00:03<00:00,  1.58it/s]

                   all        195        195      0.411      0.117      0.159     0.0911


...

                   all        195        195      0.992          1      0.995      0.812



20 epochs completed in 0.050 hours.
Optimizer stripped from logs\all_run-20240521-124737\weights\last.pt, 6.6MB
Optimizer stripped from logs\all_run-20240521-124737\weights\best.pt, 6.6MB

Validating logs\all_run-20240521-124737\weights\best.pt...
Ultralytics YOLOv8.2.18  Python-3.10.13 torch-2.1.1+cu121 CUDA:0 (NVIDIA GeForce RTX 3080 Laptop GPU, 16384MiB)
YOLOv8n-obb summary (fused): 187 layers, 3079364 parameters, 0 gradients, 8.3 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 5/5 [00:01<00:00,  3.79it/s]

                   all        195        195      0.991          1      0.995      0.812
Speed: 0.3ms preprocess, 1.2ms inference, 0.0ms loss, 2.5ms postprocess per image


ultralytics.utils.metrics.OBBMetrics object with attributes:

ap_class_index: array([ 0,  1,  2,  3,  7,  8, 10])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x00000000157235B0>
curves: []
curves_results: []
fitness: 0.8306406103475862
keys: ['metrics/precision(B)', 'metrics/recall(B)', 'metrics/mAP50(B)', 'metrics/mAP50-95(B)']
maps: array([    0.81603,     0.85872,     0.78105,     0.86127,     0.81238,     0.81238,     0.81238,     0.84235,     0.72344,     0.81238,      0.8038])
names: {0: "{0: 'resistors'}", 1: "{1: 'capacitors'}", 2: "{2: 'ceramic_cap'}", 3: "{3: 'inductors'}", 4: "{4: 'diodes'}", 5: "{5: 'mosfets'}", 6: "{6: 'transistors'}", 7: "{7: 'leds'}", 8: "{8: 'wires'}", 9: "{9: 'ics'}", 10: "{10: 'film_cap'}"}
plot: False
results_dict: {'metrics/precision(B)': 0.991195177033917, 'metrics/recall(B)': 1.0, 'metrics/mAP50(B)': 0.995, 'metrics/mAP50-95(B)': 0.8123784559417624, 'fitness': 0.83064061034758

In [9]:
# Test the model
metrics = model.val(data=TRAIN_PARAM['data'])

FileNotFoundError: 
Dataset 'models/recipes/dataset_desc.yaml' images not found ⚠️, missing path 'C:\Users\Shaheen\OneDrive - Imperial College London\Uni\CW Labs\Year 4\FYP\src\vision\datasets\current\images\val'
Note dataset download directory is 'C:\Users\Shaheen\OneDrive - Imperial College London\Uni\CW Labs\Year 4\FYP\src\vision\datasets'. You can update this in 'C:\Users\Shaheen\AppData\Roaming\Ultralytics\settings.yaml'

In [10]:
def crop_image_to_bbox(image, bbox):
    """
    Crop the image to the region within the bounding box.

    Args:
    - image (np.array): The input image.
    - bbox (tensor): The coordinates of the bounding box in xyxyxyxy format.

    Returns:
    - cropped_image (np.array): The cropped image.
    """
    # Convert tensor to numpy array and ensure it's on the CPU
    bbox = bbox.cpu().numpy()[0]

    # Compute the width and height of the bounding box
    width = int(np.linalg.norm(bbox[0] - bbox[1]))
    height = int(np.linalg.norm(bbox[1] - bbox[2]))

    # Compute the center of the bounding box
    center = np.mean(bbox, axis=0).astype(int)

    # Compute the rotation angle of the bounding box
    angle = np.degrees(np.arctan2(bbox[1, 1] - bbox[0, 1], bbox[1, 0] - bbox[0, 0]))
    rotation_matrix = cv2.getRotationMatrix2D(tuple(center), angle, 1.0)

    # Apply the rotation to the image
    rotated_image = cv2.warpAffine(image, rotation_matrix, (image.shape[1], image.shape[0]))

    # Get the bounding box in the rotated image
    x, y = center - [width // 2, height // 2]
    cropped_image = rotated_image[y:y+height, x:x+width]

    return cropped_image

### Grab component in box

In [11]:
def crop_image_to_bbox(image, bbox):
    """
    Crop the image to the region within the bounding box.

    Args:
    - image (np.array): The input image.
    - bbox (tensor): The coordinates of the bounding box in xyxyxyxy format.

    Returns:
    - cropped_image (np.array): The cropped image.
    """
    # Convert tensor to numpy array and ensure it's on the CPU
    bbox = bbox.cpu().numpy()[0]

    # Compute the width and height of the bounding box
    width = int(np.linalg.norm(bbox[0] - bbox[1]))
    height = int(np.linalg.norm(bbox[1] - bbox[2]))

    # Compute the center of the bounding box
    center = np.mean(bbox, axis=0).astype(int)
    print(center)

    # Compute the rotation angle of the bounding box
    angle = np.degrees(np.arctan2(bbox[1, 1] - bbox[0, 1], bbox[1, 0] - bbox[0, 0]))
    rotation_matrix = cv2.getRotationMatrix2D((int(center[0]), int(center[1])), angle, 1.0)

    # Apply the rotation to the image
    rotated_image = cv2.warpAffine(image, rotation_matrix, (image.shape[1], image.shape[0]))

    # Get the bounding box in the rotated image
    x, y = center - [width // 2, height // 2]
    cropped_image = rotated_image[y:y+height, x:x+width]

    return cropped_image


### Try model

In [12]:
DATA = PATHS['test']

try:
    model
    print("Loaded trained model")
except:
    # Load the best model
    # List dir and find latest run
    runs = listdir(f"./logs")
    runs.sort()
    while True:
        latest_run = runs.pop()
        if path.isdir(f"./logs/{latest_run}") and RECIPE_NAME in latest_run:
            break
    modelPath = f"./logs/{latest_run}/weights/best.pt"
    print(f"Latest run: {latest_run}, loading model from {modelPath}")
    bestModel = YOLO(modelPath).to('cuda')
    model = bestModel
    print("Loaded best model")

while cv2.waitKey(0) != ord('q'):
    cv2.destroyAllWindows()
    randomFile = random.choice(listdir(DATA))
    print(f"Random file: {randomFile}")
    results = model.predict(f"{DATA}/{randomFile}", show=True)
    cropped_img = crop_image_to_bbox(results[0].orig_img, results[0].obb.xyxyxyxy)
    cv2.imshow("Cropped Image", cropped_img)
cv2.destroyAllWindows()

Loaded trained model
Random file: obb_capacitor_330uF-16V_6.png



image 1/1 c:\Users\Shaheen\OneDrive - Imperial College London\Uni\CW Labs\Year 4\FYP\src\vision\datasets\full\current\images\test\obb_capacitor_330uF-16V_6.png: 480x640 138.5ms
Speed: 371.0ms preprocess, 138.5ms inference, 59.5ms postprocess per image at shape (1, 3, 480, 640)
[263 202]
Random file: obb_ceramic_capacitor_121_11.png

image 1/1 c:\Users\Shaheen\OneDrive - Imperial College London\Uni\CW Labs\Year 4\FYP\src\vision\datasets\full\current\images\test\obb_ceramic_capacitor_121_11.png: 480x640 151.0ms
Speed: 3.0ms preprocess, 151.0ms inference, 16.0ms postprocess per image at shape (1, 3, 480, 640)
[568 217]
Random file: obb_inductor_104_7.png

image 1/1 c:\Users\Shaheen\OneDrive - Imperial College London\Uni\CW Labs\Year 4\FYP\src\vision\datasets\full\current\images\test\obb_inductor_104_7.png: 480x640 121.5ms
Speed: 3.0ms preprocess, 121.5ms inference, 24.0ms postprocess per image at shape (1, 3, 480, 640)
[401 344]
Random file: obb_resistor_yellow_violet_black_gold_brown_470

In [6]:
cv2.destroyAllWindows()

### Draw vertices to verify direction

In [125]:
import numpy as np
def draw_bounding_box(image, bbox, bbox_format, color=(0, 255, 0), thickness=2):
    """
    Draws a bounding box on the image with an arrow indicating orientation for the xyxyxyxy format.

    Args:
    - image (np.array): The input image.
    - bbox (tensor): The bounding box coordinates.
    - bbox_format (str): The format of the bounding box ('xywhr', 'xyxy', 'xyxyxyxy', 'xyxyxyxyn').
    - color (tuple): The color of the bounding box (default is green).
    - thickness (int): The thickness of the bounding box lines (default is 2).

    Returns:
    - image_with_box (np.array): The image with the bounding box drawn.
    """
    image_with_box = image.copy()  # Make a copy of the image to avoid overwriting

    if bbox_format == 'xywhr':
        # xywhr: [x_center, y_center, width, height, rotation (in radians)]
        x_center, y_center, width, height, rotation = bbox[0]
        center = (int(x_center.item()), int(y_center.item()))
        size = (int(width.item()), int(height.item()))
        angle = np.degrees(rotation.item())

        rect = ((center[0], center[1]), (size[0], size[1]), angle)
        box = cv2.boxPoints(rect)
        box = np.int0(box)

    elif bbox_format == 'xyxy':
        # xyxy: [x_min, y_min, x_max, y_max]
        x_min, y_min, x_max, y_max = bbox[0]
        box = np.array([[x_min.item(), y_min.item()],
                        [x_max.item(), y_min.item()],
                        [x_max.item(), y_max.item()],
                        [x_min.item(), y_max.item()]], dtype=np.int0)

    elif bbox_format == 'xyxyxyxy':
        # xyxyxyxy: [[x1, y1], [x2, y2], [x3, y3], [x4, y4]]
        box = np.array(bbox[0].cpu(), dtype=np.int0)

    elif bbox_format == 'xyxyxyxyn':
        # xyxyxyxyn: normalized [[x1, y1], [x2, y2], [x3, y3], [x4, y4]]
        h, w = image.shape[:2]
        box = np.array(bbox[0].cpu() * np.array([w, h]), dtype=np.int0)

    else:
        raise ValueError(f"Unsupported bbox_format: {bbox_format}")

    # Draw the polygon
    image_with_box = cv2.polylines(image_with_box, [box], isClosed=True, color=color, thickness=thickness)

    # Draw an arrow for the xyxyxyxy format to indicate orientation
    if bbox_format == 'xyxyxyxy':
        start_point = (int(box[0][0]), int(box[0][1]))
        end_point = (int(box[1][0]), int(box[1][1]))
        image_with_box = cv2.arrowedLine(image_with_box, start_point, end_point, color, thickness, tipLength=0.3)

     # Number each vertex
    for i, (x, y) in enumerate(box):
        cv2.putText(image_with_box, str(i+1), (x, y), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), thickness, cv2.LINE_AA)

    return image_with_box

DATA = PATHS['resistors']
randomFile = random.choice(listdir(DATA))
print(f"Random file: {randomFile}")
results = model.predict(f"{DATA}/{randomFile}", show=True)

cv2.imshow(randomFile, draw_bounding_box(results[0].orig_img, results[0].obb.xyxyxyxy, 'xyxyxyxy'))
cv2.waitKey(0)
cv2.destroyAllWindows()

Random file: obb_resistor_brown_brown_black_orange_brown_red_110E3-1_8.png


C:\Users\Shaheen\AppData\Local\Temp\ipykernel_37104\1325440353.py:39: DeprecationWarning: `np.int0` is a deprecated alias for `np.intp`.  (Deprecated NumPy 1.24)
  box = np.array(bbox[0].cpu(), dtype=np.int0)


In [13]:
for result in results:
    print(result.obb.xyxyxyxy)

tensor([[[198.1221,  44.0472],
         [217.5405, 135.7320],
         [343.3661, 109.0828],
         [323.9478,  17.3980]]], device='cuda:0')
[[[198.1221466064453, 44.0472412109375], [217.54051208496094, 135.7319793701172], [343.3661193847656, 109.082763671875], [323.9477844238281, 17.398029327392578]]]
[198.1221466064453, 44.0472412109375]
